In [70]:
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns

# Preprocessing

In [71]:
data_path = 'Datasets/email.csv'
graph_path = 'nxGraph/graph_eamil'

In [72]:
df_edge = pd.read_csv(data_path, sep=',', names=['source', 'target'])

In [73]:
df_edge.head()

,source,target
0,0,0
1,0,1
2,0,101
3,0,103
4,0,146


In [74]:
# 去掉自环
df_edge = df_edge[df_edge['source'] != df_edge['target']]

In [75]:
def df2graph(df_edge, directed=False):
    """convert DataFrame to Graph(networkx)"""
    node1 = df_edge['source'].values.tolist()
    node2 = df_edge['target'].values.tolist()
    
    if directed == True:
        G = nx.DiGraph()
        G.add_edges_from(zip(node1, node2))
    else:
        G = nx.Graph()
        G.add_edges_from(zip(node1, node2))
    
    return G

In [76]:
di_graph = df2graph(df_edge, True)

In [77]:
pr = nx.pagerank(di_graph)
df_node = pd.DataFrame(pr.items(), columns=['id', 'pagerank'])

In [78]:
nodes = list(df_node['id'])

In [79]:
in_deg = list(map(lambda x: len(list(di_graph.predecessors(x))), nodes))
out_deg = list(map(lambda x: len(list(di_graph.successors(x))), nodes))

In [80]:
df_node['in_deg'] = pd.DataFrame(in_deg)
df_node['out_deg'] = pd.DataFrame(out_deg)

In [81]:
df_node.head()

,id,pagerank,in_deg,out_deg
0,0,0.001389,31,40
1,1,0.001671,50,0
2,101,0.000948,24,20
3,103,0.001620,42,32
4,146,0.000595,13,26


In [82]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [83]:
# 为边赋予权重
def weighted_graph(graph, df_node, alpha=0.7):
    weighted_graph = nx.DiGraph()
    for edge in graph.edges:
        source, target = edge
        source_out = int(df_node[df_node['id']==source]['out_deg'])
        target_in = int(df_node[df_node['id']==target]['in_deg'])
        weight = min(alpha, np.log(source_out)/np.log(target_in*source_out))
        weighted_graph.add_edge(source, target, weight=weight)
    return weighted_graph

In [84]:
w_graph = weighted_graph(di_graph, df_node)

<ipython-input-83-f2f3cf04c9df>:8: RuntimeWarning: invalid value encountered in double_scalars
  weight = min(alpha, np.log(source_out)/np.log(target_in*source_out))


In [85]:
prob_matrix = nx.adjacency_matrix(w_graph).todense().astype(np.float16)

In [86]:
di_graph = nx.DiGraph(prob_matrix, weight=prob_matrix)

In [87]:
with open(graph_path, 'wb') as f:
    pickle.dump(di_graph, f)

# Diffusion

In [88]:
def probs2state(probs):
    """
    Get the state of the node after diffusion according to the probability
    """
    return np.where(np.random.uniform(-1, 0, size=probs.shape)+probs >= 0, 1, 0)

In [89]:
def ic_diffusion(prob_matrix, state):
    for node in activated:
        probs = prob_matrix[node,:]
        state = probs2results(probs) | state

In [90]:
len(di_graph.nodes)

986

In [91]:
a1 = np.array([1,0,1,0,0,0,0,1])
a2 = np.array([1,0,0,0,1,1,0,1])

In [92]:
a1|a2

array([1, 0, 1, 0, 1, 1, 0, 1], dtype=int32)